In [1]:
from google.cloud.storage import Client
import tensorflow as tf

2024-04-23 22:54:53.208275: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 22:54:53.208303: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 22:54:53.208310: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 22:54:53.214944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os

client = Client(project="forest-lst")
bucket = "forest-lst-test-export"

records = [
    os.path.join("gs://{}".format(bucket), f.name)
    for f in client.list_blobs(bucket, prefix="ca_all_features_label")
    if ".tfrecord" in f.name
]

print("\n".join(records))

gs://forest-lst-test-export/ca_all_features_label/2016/00000.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2016/00001.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2016/00002.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2017/00000.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2017/00001.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2017/00002.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2018/00000.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2018/00001.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2018/00002.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2019/00000.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2019/00001.tfrecord
gs://forest-lst-test-export/ca_all_features_label/2019/00002.tfrecord


In [11]:
from google.protobuf import json_format
import json
from IPython.display import JSON

ds = tf.data.TFRecordDataset(records)

raw_record = next(ds.take(1).as_numpy_iterator())
example = tf.train.Example()
example.ParseFromString(raw_record)
m = json.loads(json_format.MessageToJson(example))
JSON(m)

<IPython.core.display.JSON object>

In [20]:
def decode_record(proto):
    spec = {
        "SEVERITY": tf.io.FixedLenFeature([1], dtype=tf.float32),
        "latitude": tf.io.FixedLenFeature([1], dtype=tf.float32),
        "longitude": tf.io.FixedLenFeature([1], dtype=tf.float32),
        "elevation": tf.io.FixedLenFeature([1], dtype=tf.int64),
        "is_cloudy": tf.io.VarLenFeature(dtype=tf.string),
        "prcp": tf.io.FixedLenFeature([366], dtype=tf.float32),
        "dT": tf.io.FixedLenFeature([46], dtype=tf.float32)
    }

    ex = tf.io.parse_single_example(proto, spec)

    return ex

ds_dec = ds.map(decode_record)